In [8]:
import os
import pandas as pd
import re
import numpy as np
import datetime

In [9]:
bilibili_list = [730603253025,730901991727,730903103301]
def bilibili_taoke_divide(sku_id,bilibili_list,file_name):
    if sku_id in bilibili_list and ("定向计划" in file_name or "自主推广-商品维度-推广概览-数据分布" in file_name):
        return "B站"
    else:
        return "taoke"

In [10]:
itemList = ["塞德思","灵的融","希纳露","百热朗","健眠计划","S+","琵力消","VL"]
def tag_add(itemName,itemLists):
    
        for item in itemLists:
            if item in itemName:
                return item
            else:
                continue
        return itemName

In [11]:
def get_liuliang_date(xls_file_name):
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    liuliang_date = re.findall(date_pattern,xls_file_name)[0]
    return liuliang_date

In [12]:
xls_list=os.listdir(r"D:\盐野义\月报\渠道内容更新")
liuliang_data = pd.read_csv("D:/盐野义/月报/渠道内容更新/"+xls_list[0])
liuliang_data["日期"] = pd.to_datetime(get_liuliang_date(xls_list[0]))
liuliang_data["渠道"] = liuliang_data["商品ID"].apply(lambda x:bilibili_taoke_divide(sku_id=x,bilibili_list=bilibili_list,file_name = xls_list[0]))

In [13]:
for _data in xls_list[1:]:
    liuliang_data_sub = pd.read_csv("D:/盐野义/月报/渠道内容更新/"+_data)
    liuliang_data_sub["日期"] = pd.to_datetime(get_liuliang_date(_data))
    liuliang_data_sub["渠道"] = liuliang_data_sub["商品ID"].apply(lambda x:bilibili_taoke_divide(sku_id=x,bilibili_list=bilibili_list,file_name = _data))
    liuliang_data = pd.concat([liuliang_data,liuliang_data_sub],axis=0,ignore_index=True)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_1808\2156170807.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  liuliang_data = pd.concat([liuliang_data,liuliang_data_sub],axis=0,ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1808\2156170807.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  liuliang_data = pd.concat([liuliang_data,liuliang_data_sub],axis=0,ignore_index=True)


In [14]:
from numpy import int64
liuliang_data["月份"] = liuliang_data["日期"].dt.month
liuliang_data.dropna(subset=["商品名称"],inplace=True)
liuliang_data["商品类型"] = liuliang_data["商品名称"].apply(lambda x:tag_add(itemName=x,itemLists=itemList))
liuliang_data["点击人数(即进店人数)"] = liuliang_data["点击人数(即进店人数)"].astype(int64)


In [16]:
liuliang_pivot_table = liuliang_data.pivot_table(values=["点击人数(即进店人数)","付款支出费用","结算金额","付款人数"],index=["月份","渠道","商品类型"],aggfunc=sum)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1808\3921194608.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  liuliang_pivot_table = liuliang_data.pivot_table(values=["点击人数(即进店人数)","付款支出费用","结算金额","付款人数"],index=["月份","渠道","商品类型"],aggfunc=sum)


In [17]:
liuliang_pivot_table.to_excel("D:/盐野义/月报/2024年6月/淘客&b站渠道.xlsx")